In [1]:
#Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise.
!pip install scikit-Surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-Surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357284 sha256=4acdbaccd1cc3bc31dda12bcea7ff0f02f0d5b0a643810daf24c99f1c6d562a4
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-Surprise


In [3]:
#Виведіть перші 5 рядків завантаженого датасету.
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')

print("Перші 5 рядків датасету з поясненням: ")
for row in data.raw_ratings[:5]:
  user_id, movie_id, rating, timestamp = row
  print(f"Користувач {user_id} оцінив фільм {movie_id} на {rating} балів (часова позначка: {timestamp})")

Перші 5 рядків датасету з поясненням: 
Користувач 196 оцінив фільм 242 на 3.0 балів (часова позначка: 881250949)
Користувач 186 оцінив фільм 302 на 3.0 балів (часова позначка: 891717742)
Користувач 22 оцінив фільм 377 на 1.0 балів (часова позначка: 878887116)
Користувач 244 оцінив фільм 51 на 2.0 балів (часова позначка: 880606923)
Користувач 166 оцінив фільм 346 на 1.0 балів (часова позначка: 886397596)


In [4]:
#Реалізуйте два алгоритми для рекомендаційної системи на основі цього датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.
#Використайте крос-валідацію для підбору оптимальних параметрів для обох алгоритмів.
trainset, testset = train_test_split(data, test_size=0.2)

param_grid_svd = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV (SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV (KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

In [5]:
#Оберіть найкращий алгоритм на основі середньої абсолютної помилки (МАЕ).
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print (f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print (f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

if grid_search_svd.best_score['mae'] <grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

Найкращий MAE для SVD: 0.7239883338895425
Найкращий MAE для KNNBasic: 0.802672357872433
Вибраний алгоритм: SVD


In [7]:
#Виведи рекомендації (10 фільмів) для конкретного користувача.
best_model.fit (trainset)

user_id= str(244)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print (f"Кількість рецензій користувача {user_id}: {len(user_ratings)} ")

all_items = set (trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ 10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions [:10]:
    print (f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 244: 182 
Топ 10 фільмів, рекомендованих для користувача:
Фільм 56 з прогнозованим рейтингом 5.00
Фільм 474 з прогнозованим рейтингом 4.96
Фільм 475 з прогнозованим рейтингом 4.91
Фільм 127 з прогнозованим рейтингом 4.87
Фільм 1142 з прогнозованим рейтингом 4.86
Фільм 408 з прогнозованим рейтингом 4.86
Фільм 320 з прогнозованим рейтингом 4.79
Фільм 275 з прогнозованим рейтингом 4.78
Фільм 178 з прогнозованим рейтингом 4.76
Фільм 150 з прогнозованим рейтингом 4.76


In [ ]:
#Висновок: У даній лабораторній роботі було розглянуто створення рекомендаційної системи на основі датасету MovieLens 100k, використовуючи бібліотеку Surprise.
#Основними етапами роботи були: Завантаження датасету MovieLens 100k та виведення перших 5 рецензій з поясненням структури даних.
#Реалізація двох алгоритмів рекомендаційної системи: SVD (сингулярний розклад матриці), KNNBasic (метод найближчих сусідів).
#Підбір оптимальних параметрів для кожного з алгоритмів за допомогою крос-валідації (GridSearchCV).
#Для обох алгоритмів були підібрані оптимальні значення параметрів, зокрема кількість факторів, швидкість навчання та регуляризація для SVD, а також кількість сусідів і тип подібності для KNNBasic.
#Оцінка алгоритмів на основі метрики MAE (середня абсолютна помилка).
#Було визначено, що алгоритм SVD демонструє кращий результат (MAE: 0.7239) у порівнянні з KNNBasic (MAE: 0.8026), тому його було вибрано як основний для рекомендацій.
#Побудова рекомендацій для користувача: на основі навченого алгоритму SVD було створено список з 10 фільмів, які не оцінював користувач, але мають високий прогнозований рейтинг.

#Загальні висновки:
#Алгоритм SVD показав кращі результати для даного датасету, оскільки враховує латентні фактори і демонструє більш низькі значення помилки.
#Метод крос-валідації дозволив ефективно підібрати оптимальні параметри для обох алгоритмів.
#Рекомендаційні системи на основі методів факторизації, таких як SVD, є ефективними у прогнозуванні уподобань користувачів, що робить їх придатними для реальних додатків, таких як онлайн-платформи для фільмів чи товарів.

#Посилання на гітхаб: https://github.com/nastiadzen/Machine-learning